In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [2]:
def read_url(url):
    response = requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,'lxml')
    return soup

# All apartments pages

In [3]:
url_list=[]

for i in range(1,355):
    url_list.append('https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/{}?rent_type=3'.format(i))

In [4]:
url_list[:5]

['https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/1?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/2?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/3?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/4?rent_type=3',
 'https://sa.aqar.fm/%D8%B4%D9%82%D9%82-%D9%84%D9%84%D8%A5%D9%8A%D8%AC%D8%A7%D8%B1/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/5?rent_type=3']

# Scrape apartment urls

In [5]:
apartments_list = []
for url in tqdm(url_list):
    for page in read_url(url).find_all('a',{'class':'listTitle'}):
        apartments_list.append('https://sa.aqar.fm' + page.get("href"))
len(apartments_list)

  0%|          | 0/354 [00:00<?, ?it/s]

7080

In [6]:
apartments_list[:5]

['https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-ظهرة-لبن/شارع-ينبع-حي-ظهرة-لبن-الرياض-الرياض-2526973',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الرمال/شارع-أبي-العلاء-الواسطي-حي-الرمال-الرياض-الرياض-3119941',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-السلام/شارع-الكهف-حي-السلام-الرياض-الرياض-3440789',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-طويق/شارع-احمد-بن-الخطاب-حي-طويق-الرياض-الرياض-3623160',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-النهضة/شارع-عبدالله-بن-سويلم-حي-النهضة-الرياض-الرياض-3567421']

write list to excel file

In [7]:
pd.Series(apartments_list).to_excel('apartments_list2.xlsx')

Or write it to a .txt file

In [8]:
textfile = open("apartments_list2.txt", "w")
for element in apartments_list:
    textfile.write(element + "\n")
textfile.close()

Read .txt file

In [9]:
with open('apartments_list2.txt','r') as f:
    apt_urls = f.read().splitlines()

In [10]:
len(apt_urls)

7080

In [11]:
apt_urls[:5]

['https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-ظهرة-لبن/شارع-ينبع-حي-ظهرة-لبن-الرياض-الرياض-2526973',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الرمال/شارع-أبي-العلاء-الواسطي-حي-الرمال-الرياض-الرياض-3119941',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-السلام/شارع-الكهف-حي-السلام-الرياض-الرياض-3440789',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-طويق/شارع-احمد-بن-الخطاب-حي-طويق-الرياض-الرياض-3623160',
 'https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-النهضة/شارع-عبدالله-بن-سويلم-حي-النهضة-الرياض-الرياض-3567421']

In [12]:
def convert(page):
    price_tag = page.find('span', {'class':'listingPagePrice'})
    price = float(price_tag.text.split()[4].replace(",",""))
    
    #if price_tag.text.split()[3][0]=='ش':
    #    return price * 12
    #elif price_tag.text.split()[3][0]=='ي':
    #    return price * 355
    #else:
    #    return price
    return price, price_tag.text.split()[3]

# Scrape pages details

In [ ]:
PRICE_list = []
DISTRICT_list = []
BED_list = []
BATH_list = []
LIVING_List = []
AGE_list = []
FLOOR_list = []
TARGET_list = []
REGION_list = []
DURATION_list = []
urlapt = []

IMAGES_list = []

FURNISHED_list = []
KITCHEN_list = []
ANNEXE_list = []
PARKING_list = []
ELEVATOR_list = []
AC_list = []

R = ['وسط الرياض','جنوب الرياض','شمال الرياض','شرق الرياض','غرب الرياض']

for url in tqdm(apt_urls):
    try:
        # get page, price, neighborhood
        page = read_url(url)
        PRICE, DURATION = convert(page)

        DISTRICT = np.NaN
        div = page.find('div', class_='tree treeMargin')
        for link in div.find_all('a'):
            if(link.text.startswith('حي')):
                DISTRICT = link.text
            if any(r in link.text for r in R):
                REGION = link.text
        

        # main table
        table = page.find('table',{"class":"dc-detailsTable"}).find_all('tr')

        BED = np.NaN
        BATH = np.NaN
        AGE = np.NaN
        LIVING = np.NaN

        for tr in table:
            td = tr.find_all('td')
            row = [i.text for i in td]
            if row[1] == 'غرف النوم':
                BED = row[0]
            if row[1] == 'دورات مياه':
                BATH = row[0]
            if row[1] == 'الصالات':
                LIVING = row[0]
            if row[1] == 'عمر العقار':
                AGE = row[0].strip().split()[0]
                if AGE == 'جديد':
                    AGE = 0
            if row[1] == 'الدور':
                FLOOR = row[0]
            if row[1] == 'الفئة':
                if row[0] == "عوائل":
                    TARGET = "Families"
                if row[0] == "عزاب":
                    TARGET = "Bachelors"


        PRICE_list.append(PRICE)
        BED_list.append(BED)
        BATH_list.append(BATH)
        LIVING_List.append(LIVING)
        AGE_list.append(AGE)
        DISTRICT_list.append(DISTRICT)
        FLOOR_list.append(FLOOR)
        TARGET_list.append(TARGET)
        REGION_list.append(REGION)
        DURATION_list.append(DURATION)
        urlapt.append(url)

        # extra table
        FURNISHED, KITCHEN, ANNEXE, PARKING, ELEVATOR, AC = 0,0,0,0,0,0

        extraTable = page.find('table', {"class": "dc-extraDetailsTable"}).find_all('tr')

        for tr2 in extraTable:
            td2 = tr2.find_all('td')
            row2 = [i.text for i in td2]
            if 'مؤثثة' in row2:
                FURNISHED = 1
            if 'مطبخ' in row2:
                KITCHEN = 1
            if 'ملحق' in row2:
                ANNEXE = 1
            if 'مدخل سيارة' in row2:
                PARKING = 1
            if 'مصعد' in row2:
                ELEVATOR = 1
            if 'مكيفات' in row2:
                AC = 1


        FURNISHED_list.append(FURNISHED)
        KITCHEN_list.append(KITCHEN)
        ANNEXE_list.append(ANNEXE)
        PARKING_list.append(PARKING)
        ELEVATOR_list.append(ELEVATOR)
        AC_list.append(AC)        


        # image count
        try:            
            images = 0
            b = len(page.find('div', {'class':'listingImgContainer'}).findAll("img",{"class":"listingPageImg"}))
            c = len(page.find('div', {'class':'listingImgContainer'}).findAll("img",{"class":"listingPageImg hide"}))
            images = b-c
            IMAGES_list.append(images)
        except Exception:
            IMAGES_list.append(0)        


    except Exception as e:
        print(e, "->", url)

  0%|          | 0/7080 [00:00<?, ?it/s]

'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شمال-الرياض/حي-النرجس/شارع-الديوان-حي-النرجس-الرياض-الرياض-3626283
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/غرب-الرياض/حي-العريجاء-الغربية/شارع-الشويرف-حي-العريجاء-الغربية-الرياض-الرياض-3583983
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-اليرموك/شارع-عتود-حي-اليرموك-الرياض-الرياض-3636131
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-اليرموك/شارع-الهملة-حي-اليرموك-الرياض-الرياض-3644657
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-اليرموك/شارع-الهملة-حي-اليرموك-الرياض-الرياض-3644657
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/شرق-الرياض/حي-الخليج/شارع-الضائن-حي-الخليج-الرياض-الرياض-3641468
'NoneType' object has no attribute 'text' -> https://sa.aqar.fm/شقق-للإيجار/الرياض/وس

In [ ]:
dataset = pd.DataFrame({
    "DISTRICT": DISTRICT_list,
    "REGION": REGION_list,
    "PRICE": PRICE_list,
    "DURATION": DURATION_list,
    "BED": BED_list,
    "BATH": BATH_list,
    "LIVING": LIVING_List,
    "AGE": AGE_list,
    "IMAGES": IMAGES_list,
    "FLOOR": FLOOR_list,
    "TARGET": TARGET_list,
    "FURNISHED": FURNISHED_list,
    "KITCHEN": KITCHEN_list,
    "ANNEXE": ANNEXE_list,
    "PARKING": PARKING_list,
    "ELEVATOR": ELEVATOR_list,
    "AC": AC_list,
    "url": urlapt
})
dataset[:40]

In [ ]:
dataset.info()

In [ ]:
dataset.to_excel("Aqarat3.xlsx")

Read excel file

In [ ]:
d=pd.read_excel("Aqarat3.xlsx")
d.drop("Unnamed: 0",axis=1,inplace=True)
d

In [ ]:
d.REGION.value_counts()